In [22]:
import scipy
import scipy.io
from scipy.optimize import curve_fit
from scipy.optimize import least_squares
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import statistics
import math
import time
import itertools
from tqdm import trange
from datetime import date
import pickle
import pandas as pd
import sys
import os
import functools

In [23]:
TE_DATA = np.linspace(8, 512, 64) #ms units

noise_opt = True
SNR_value = 100

T21_center = 50
T22_center = 100

c1 = 0.2
c2 = 0.8

T21_range = np.arange(T21_center-30, T21_center+30+.001, 1)
T22_range = np.arange(T22_center-30, T22_center+30+.001, 1)

mT21, mT22 = np.meshgrid(T21_range, T22_range)

repetitions = 1000

lamb_oi = 0.1
lamb_oi2 = 0.2

RRSS = r'RSS$^{1/2}$'
T21_label = r'$T_{21}$'
T22_label = r'$T_{22}$'
c1_label = r'$c_1$'
c2_label = r'$c_2$'
lamb_lab = r'$\lambda$'


if noise_opt:
    noise_info = f"SNR = {SNR_value}"
else:
    noise_info = "noiseless"

In [24]:
def G_biX(t, con_1, con_2, tau_1, tau_2): 
    signal = con_1*np.exp(-t/tau_1) + con_2*np.exp(-t/tau_2)
    return signal

def G_tilde(lam):
    def Gt_lam(t, con1, con2, tau1, tau2):
        return np.append(G_biX(t, con1, con2, tau1, tau2), [lam*con1, lam*con2, lam*tau1/100, lam*tau2/100])
    return Gt_lam

def J_biX(TE, d1, d2, T21, T22):
    #Returns the Jacobian of our 4 parameter, 2D problem
    dd1 = np.exp(-TE/T21)
    dd2 = np.exp(-TE/T22)
    dT21 = (d1*TE/T21**2)*np.exp(-TE/T21)
    dT22 = (d2*TE/T22**2)*np.exp(-TE/T22)
    
    jacobian = np.stack((dd1, dd2, dT21, dT22), axis = -1)
    return jacobian

In [25]:
def add_noise(data, SNR):
    #returns a noised vector of data using the SNR given
    sigma = 1/SNR #np.max(np.abs(data))/SNR
    noise = np.random.normal(0,sigma,data.shape)
    noised_data = data + noise
    return noised_data


def format_fn(tick_val, tick_pos, labels = ""):
    if int(tick_val) < len(labels):
        return labels[int(tick_val)]
    else:
        return ''

def calculate_RSS(input, func, popt, data):
    est_curve = func(input, *popt)
    RSS = np.sum((est_curve - data)**2)
    
    return RSS

def calculate_regRSS(input, func, popt, lamb, data):
    est_curve = func(input, *popt)
    RSS = np.sum((est_curve - data)**2)
    weights = np.array([1,1,1/100,1/100])
    reg = np.sum(lamb**2 *(popt*weights)**2)
    
    return RSS+reg

# Using the Curve Fit Full Output

In [26]:
true_signal = G_biX(TE_DATA, c1, c2, T21_center, T22_center)
noisey_sig = add_noise(true_signal, SNR_value)
data_tilde = np.append(noisey_sig, [0,0,0,0])
p_init = [0.3, 0.7, 20, 80]

lamb_oi = 0

popt_nonreg, _, info_nonreg, _, _ = curve_fit(G_tilde(lamb_oi), TE_DATA, data_tilde, bounds = (0, [2,2,100,300]), p0=p_init, max_nfev = 4000, full_output=True)
resid_nonreg = info_nonreg['fvec']

RSS_nonreg_simple = calculate_RSS(TE_DATA, G_biX, popt_nonreg, noisey_sig)
RSS_nonreg_reg = calculate_regRSS(TE_DATA, G_biX, popt_nonreg, lamb_oi, noisey_sig)
RSS_nonreg_sum = np.sum(resid_nonreg**2)

print(f"Nonreg results for parameter estimation")
print(f"RSS with simple RSS calculation = {RSS_nonreg_simple: 0.5f}")
print(f"RSS with regularized RSS calculation = {RSS_nonreg_reg: 0.5f}")
print(f"RSS with from residuals of cf = {RSS_nonreg_sum: 0.5f}")

Nonreg results for parameter estimation
RSS with simple RSS calculation =  0.00721
RSS with regularized RSS calculation =  0.00721
RSS with from residuals of cf =  0.00721


In [27]:
lamb_oi = 0.1

popt_nonreg, _, info_nonreg, _, _ = curve_fit(G_tilde(lamb_oi), TE_DATA, data_tilde, bounds = (0, [2,2,100,300]), p0=p_init, max_nfev = 4000, full_output=True)
resid_nonreg = info_nonreg['fvec']

RSS_nonreg_simple = calculate_RSS(TE_DATA, G_biX, popt_nonreg, noisey_sig)
RSS_nonreg_reg = calculate_regRSS(TE_DATA, G_biX, popt_nonreg, lamb_oi, noisey_sig)
RSS_nonreg_sum = np.sum(resid_nonreg**2)

print(f"Nonreg results for parameter estimation with ")
print(f"RSS with simple RSS calculation = {RSS_nonreg_simple: 0.5f}")
print(f"RSS with regularized RSS calculation = {RSS_nonreg_reg: 0.5f}")
print(f"RSS with from residuals of cf = {RSS_nonreg_sum: 0.5f}")

Nonreg results for parameter estimation with 
RSS with simple RSS calculation =  0.00820
RSS with regularized RSS calculation =  0.02603
RSS with from residuals of cf =  0.02603
